In [ ]:
%pip install openai langchain google-api-python-client

In [9]:
# Agents - LLMs

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import AzureOpenAI

# The language model we're going to use to control the agent.
llm = AzureOpenAI(temperature=0, deployment_name="textdavinci003")

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["google-search", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Let's test it out!
agent.run(
    "What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?<|im_end|>"
)



> Entering new  chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: google_search
Action Input: "high temperature in SF yesterday"
Observation: Weather reports from the last weeks in San Francisco with highs and lows. High: 69.8ºf @12:00 PM Low: 57.2ºf @4:10 AM Approx. Precipitation / Rain Total: in. Time (PDT), Temp. (ºf), Humidity (%), Dew Point ( ... San Francisco Weather Forecasts. Weather Underground provides local ... Almanac for June 18, 2023. Forecast. Average *. Record. Temperature. High. Sep 6, 2022 ... Bay Area heat wave: More cities break all-time high temps Tuesday ... recorded an astounding 116 degrees, which it also recorded yesterday. Sep 6, 2022 ... "Early morning temperatures in the East Bay and higher terrain are running 5 to 10 degrees higher today than they were yesterday," Brooke ... Jun 11, 2019 ... San Francisco International Airport yesterday reached its highest temperature ever recorded in the month 

'1.1025763550530143'

In [11]:
# Agents - Chat models

from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI

# First, let's load the language model we're going to use to control the agent.
chat = AzureChatOpenAI(temperature=1, deployment_name="gpt35")

# Note: llm-math might randomly fail when using gpt model due to this known issue
# https://github.com/hwchase17/langchain/issues/3071
# llm = AzureOpenAI(temperature=0.1, deployment_name="gpt35")

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = AzureOpenAI(temperature=0, deployment_name="textdavinci003")
tools = load_tools(["google-search", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Let's test it out!
agent.run(
    "What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?"
)



> Entering new  chain...
Thought: The first part of the question can be answered with a google search. The second part will require a calculator.

Action:
```
{
  "action": "google_search",
  "action_input": "San Francisco yesterday high temperature fahrenheit"
}
```


Observation: Weather reports from the last weeks in San Francisco with highs and lows. ... Maximum temperature yesterday: 68 °F (at 11:56 am) San Francisco, San Francisco International Airport ... Weather, Sky Cond. Temperature (ºF), Relative Humidity, Wind Chill (°F), Heat San Francisco Weather Forecasts. ... TodayMon 06/19 High 64 °F ... Almanac for June 18, 2023. Forecast. Average *. Record. Temperature. High. 67 °F. 72 °F. Weather reports from the last weeks in San Francisco Bay Area with highs and lows. ... Maximum temperature yesterday: 68 °F (at 11:56 am) Temperature (°F), Actual, Historic Avg. Record. High Temp, 57, 71.9, 91. Low Temp, 57, 53.8, 45. Day Average Temp, 57, 62.9, - ... Sep 6, 2022 ... Kentfield al

'1.101914010425215'

In [3]:
# Memory - LLMs

import langchain

from langchain import ConversationChain
from langchain.llms import AzureOpenAI

# Enable debug
langchain.debug = True

llm = AzureOpenAI(
    temperature=1,
    model_name="gpt-35-turbo",
    deployment_name="gpt35",
    model_kwargs={"stop": ["Human:", "AI"]},
)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.run("Hi there! I'm Phong")
conversation.run("Can you remind me what my name is?")
conversation.run("What is the weather in New York now?")

[chain/start] [1:RunTypeEnum.chain:ConversationChain] Entering Chain run with input:
{
  "input": "Hi there! I'm Phong",
  "history": ""
}
[llm/start] [1:RunTypeEnum.chain:ConversationChain > 2:RunTypeEnum.llm:AzureOpenAI] Entering LLM run with input:
{
  "prompts": [
    "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n\nHuman: Hi there! I'm Phong\nAI:"
  ]
}
[llm/end] [1:RunTypeEnum.chain:ConversationChain > 2:RunTypeEnum.llm:AzureOpenAI] [478.043ms] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Hello Phong! I'm glad to meet you!\n",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 64,
      "total_tokens": 75,
 

' Sorry, I do not know the weather in New York at this moment. But I can search on the internet for you if you like.\n\n'

In [4]:
# Debug buffer

print(conversation.memory.buffer)

Human: Hi there! I'm Phong
AI:  Hello Phong! I'm glad to meet you!

Human: Can you remind me what my name is?
AI:   Your name is Phong!


Human: What is the weather in New York now?
AI:  Sorry, I do not know the weather in New York at this moment. But I can search on the internet for you if you like.




In [6]:
# Memory - Chat models

import langchain

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory

# Enable debug
langchain.debug = True

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "The following is a friendly conversation between a human and an AI. The AI is talkative and "
            "provides lots of specific details from its context. If the AI does not know the answer to a "
            "question, it truthfully says it does not know."
        ),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

chat = AzureChatOpenAI(
    temperature=1, model_name="gpt-35-turbo", deployment_name="gpt35"
)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=chat)

conversation.run(input="Hi there! I'm Phong")
conversation.run(input="Can you remind me what my name is?")
conversation.run(input="What is the weather in New York now?")

[chain/start] [1:RunTypeEnum.chain:ConversationChain] Entering Chain run with input:
{
  "input": "Hi there! I'm Phong",
  "history": []
}
[llm/start] [1:RunTypeEnum.chain:ConversationChain > 2:RunTypeEnum.llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\nHuman: Hi there! I'm Phong"
  ]
}
[llm/end] [1:RunTypeEnum.chain:ConversationChain > 2:RunTypeEnum.llm:AzureChatOpenAI] [669.1049999999999ms] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello Phong! It's nice to meet you. How can I assist you today?",
        "generation_info": null,
        "message": {
          "content": "Hello Phong! It's nice to meet you. How can I assist you today?",
          "additional_kwargs": {},
          "examp

"I'm sorry, but I don't have access to real-time weather data. However, according to the latest weather reports, New York is experiencing a mix of sun and clouds with temperatures in the mid-60s Fahrenheit. Would you like me to look up more information about the weather in New York?"

In [7]:
# Debug buffer

print(conversation.memory.buffer)

[HumanMessage(content="Hi there! I'm Phong", additional_kwargs={}, example=False), AIMessage(content="Hello Phong! It's nice to meet you. How can I assist you today?", additional_kwargs={}, example=False), HumanMessage(content='Can you remind me what my name is?', additional_kwargs={}, example=False), AIMessage(content='Certainly! Your name is Phong.', additional_kwargs={}, example=False), HumanMessage(content='What is the weather in New York now?', additional_kwargs={}, example=False), AIMessage(content="I'm sorry, but I don't have access to real-time weather data. However, according to the latest weather reports, New York is experiencing a mix of sun and clouds with temperatures in the mid-60s Fahrenheit. Would you like me to look up more information about the weather in New York?", additional_kwargs={}, example=False)]
